In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import warnings; warnings.simplefilter('ignore')
from get_movie_features import movie_feature
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

In [2]:
credits_ = r'credits.csv'
keywords = r'keywords.csv'
links_small = r'links_small.csv'
movies_metadata = r'movies_metadata.csv'


smd = movie_feature(movies_metadata, links_small, credits_, keywords)
smd.head(10)

,id,movieId,title,genres,description,keywords,cast,director,spoken_languages,production_companies,production_countries,popularity,year,vote_average,vote_count,wr
0,862,1,Toy Story,"[Animation, Comedy, Family]","Led by Woody, Andy's toys live happily in his ...","[jealousi, toy, boy, friendship, friend, rival...","[tomhanks, timallen, donrickles]",johnlasseter,[English],[Pixar Animation Studios],[United States of America],21.946943,1995,7,5415,6.869770
1,8844,2,Jumanji,"[Adventure, Fantasy, Family]",When siblings Judy and Peter discover an encha...,"[boardgam, disappear, basedonchildren'sbook, n...","[robinwilliams, jonathanhyde, kirstendunst]",joejohnston,"[English, Français]","[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],17.015539,1995,6,2413,5.884891
2,15602,3,Grumpier Old Men,"[Romance, Comedy]",A family wedding reignites the ancient feud be...,"[fish, bestfriend, duringcreditssting]","[waltermatthau, jacklemmon, ann-margret]",howarddeutch,[English],"[Warner Bros., Lancaster Gate]",[United States of America],11.712900,1995,6,92,5.376968
3,31357,4,Waiting to Exhale,"[Comedy, Drama, Romance]","Cheated on, mistreated and stepped on, the wom...","[basedonnovel, interracialrelationship, single...","[whitneyhouston, angelabassett, lorettadevine]",forestwhitaker,[English],[Twentieth Century Fox Film Corporation],[United States of America],3.859495,1995,6,34,5.299755
4,11862,5,Father of the Bride Part II,[Comedy],Just when George Banks has recovered from his ...,"[babi, midlifecrisi, confid, age, daughter, mo...","[stevemartin, dianekeaton, martinshort]",charlesshyer,[English],"[Sandollar Productions, Touchstone Pictures]",[United States of America],8.387519,1995,5,173,5.175099
5,949,6,Heat,"[Action, Crime, Drama, Thriller]","Obsessive master thief, Neil McCauley leads a ...","[robberi, detect, bank, obsess, chase, shoot, ...","[alpacino, robertdeniro, valkilmer]",michaelmann,"[English, Español]","[Regency Enterprises, Forward Pass, Warner Bros.]",[United States of America],17.924927,1995,7,1886,6.671675
6,11860,7,Sabrina,"[Comedy, Romance]",An ugly duckling having undergone a remarkable...,"[pari, brotherbrotherrelationship, chauffeur, ...","[harrisonford, juliaormond, gregkinnear]",sydneypollack,"[Français, English]","[Paramount Pictures, Scott Rudin Productions, ...","[Germany, United States of America]",6.677277,1995,6,141,5.430061
7,45325,8,Tom and Huck,"[Action, Adventure, Drama, Family]","A mischievous young boy, Tom Sawyer, witnesses...",[],"[jonathantaylorthomas, bradrenfro, rachaelleig...",peterhewitt,"[English, Deutsch]",[Walt Disney Pictures],[United States of America],2.561161,1995,5,45,5.221890
8,9091,9,Sudden Death,"[Action, Adventure, Thriller]",International action superstar Jean Claude Van...,"[terrorist, hostag, explos, vicepresid]","[jean-claudevandamme, powersboothe, dorianhare...",peterhyams,[English],"[Universal Pictures, Imperial Entertainment, S...",[United States of America],5.231580,1995,5,174,5.174811
9,710,10,GoldenEye,"[Adventure, Action, Thriller]",James Bond must unmask the mysterious head of ...,"[cuba, falselyaccus, secretident, computerviru...","[piercebrosnan, seanbean, izabellascorupco]",martincampbell,"[English, Pусский, Español]","[United Artists, Eon Productions]","[United Kingdom, United States of America]",14.686036,1995,6,1194,5.798701


In [3]:
smd.columns

Index(['id', 'movieId', 'title', 'genres', 'description', 'keywords', 'cast',
       'director', 'spoken_languages', 'production_companies',
       'production_countries', 'popularity', 'year', 'vote_average',
       'vote_count', 'wr'],
      dtype='object')

In [4]:
flattened_genres = [e for sublist in smd['genres'] for e in sublist]
flattened_keywords = [e for sublist in smd['keywords'] for e in sublist]
flattened_spoken_languages = [e for sublist in smd['spoken_languages'] for e in sublist]
flattened_production_companies = [e for sublist in smd['production_companies'] for e in sublist]
flattened_production_countries = [e for sublist in smd['production_countries'] for e in sublist]

In [5]:
genres = LabelEncoder().fit(flattened_genres)

In [6]:
keywords = LabelEncoder().fit(flattened_keywords)

In [7]:
smd['director'] = preprocessing.LabelEncoder().fit_transform(smd['director'])

In [8]:
sp_lan = LabelEncoder().fit(flattened_spoken_languages)

In [9]:
pro_com = LabelEncoder().fit(flattened_production_companies)

In [10]:
pro_coun = LabelEncoder().fit(flattened_production_countries)

In [11]:
genres_enc = [list(genres.transform(sublist)) for sublist in smd['genres']]

keyword_enc = [list(keywords.transform(sublist)) for sublist in smd['keywords']]

spoken_languages_enc = [list(sp_lan.transform(sublist)) for sublist in smd['spoken_languages']]

production_companies_enc = [list(pro_com.transform(sublist)) for sublist in smd['production_companies']]

production_countries_enc = [list(pro_coun.transform(sublist)) for sublist in smd['production_countries']]

In [12]:
genres_enc

[[2, 3, 7],
 [1, 8, 7],
 [14, 3],
 [3, 6, 14],
 [3],
 [0, 4, 6, 17],
 [3, 14],
 [0, 1, 6, 7],
 [0, 1, 17],
 [1, 0, 17],
 [3, 6, 14],
 [3, 11],
 [7, 2, 1],
 [10, 6],
 [0, 1],
 [6, 4],
 [6, 14],
 [4, 3],
 [4, 3, 1],
 [0, 3, 4],
 [3, 17, 4],
 [6, 17],
 [0, 1, 4, 17],
 [6, 8, 15, 17],
 [6, 14],
 [6],
 [3, 6, 7],
 [6, 14],
 [8, 15, 1],
 [6, 4],
 [6, 4],
 [15, 17, 13],
 [8, 6, 3, 7],
 [10, 6, 14],
 [6],
 [1, 10, 6, 7],
 [3, 7, 14],
 [3, 6, 14],
 [6],
 [6, 18],
 [0, 4, 6, 10],
 [6, 14],
 [0, 8],
 [8, 6, 3, 17],
 [6, 14],
 [4, 13, 17],
 [1, 2, 6, 7],
 [6, 14],
 [6, 4, 17],
 [3, 14],
 [6, 9],
 [0, 1, 3, 7],
 [6],
 [3, 6, 14],
 [3, 6, 14],
 [17, 6, 13],
 [1, 7, 8],
 [6, 17],
 [12, 6, 7],
 [3],
 [3, 14],
 [3],
 [0, 15],
 [3, 14],
 [3],
 [11, 0, 17, 4],
 [0, 17, 14],
 [3, 6, 14],
 [6, 10],
 [6, 14],
 [11, 15],
 [5],
 [6, 17],
 [6, 17],
 [7, 6],
 [6, 4],
 [6, 3],
 [14, 6],
 [],
 [6, 14],
 [0, 6],
 [1, 3, 7],
 [3],
 [4, 6, 17],
 [6, 11, 17, 14],
 [3, 11, 14],
 [3, 6, 14],
 [0, 1, 6, 17],
 [3],
 [6],

In [13]:
smd['genres']

0                             [Animation, Comedy, Family]
1                            [Adventure, Fantasy, Family]
2                                       [Romance, Comedy]
3                                [Comedy, Drama, Romance]
4                                                [Comedy]
                              ...                        
9214                                              [Drama]
9215                                  [Thriller, Romance]
9216                 [Adventure, Drama, History, Romance]
9217    [Action, Adventure, Drama, Horror, Science Fic...
9218                                 [Documentary, Music]
Name: genres, Length: 9219, dtype: object

In [14]:
def cosine_similarity_genres(genres_enc1, genres_enc2):
  genres_vec1 = np.array(genres_enc1)
  genres_vec2 = np.array(genres_enc2)
  return np.dot(genres_vec1, genres_vec2) / (np.linalg.norm(genres_vec1) * np.linalg.norm(genres_vec2))

In [15]:
movie1_genres_enc = genres_enc[3]  
movie2_genres_enc = genres_enc[1]
genres_similarity = cosine_similarity_genres(movie1_genres_enc, movie2_genres_enc)
print("Cosine similarity between movies based on genres:", genres_similarity)

Cosine similarity between movies based on genres: 0.8989288555177889


In [16]:
def cosine_similaritys(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [17]:
genres_vec_1 = np.array(movie1_genres_enc)
genres_vec_2 = np.array(movie2_genres_enc)
genres_vec_1 = genres_vec_1.reshape(1, -1)
genres_vec_2 = genres_vec_2.reshape(1, -1)
genre_sim = cosine_similarity(genres_vec_1,genres_vec_2)

In [18]:
genre_sim

array([[0.89892886]])

In [28]:
train_df = pd.read_csv('train_set.csv')

In [29]:
train_df

,userId,movieId,rating,timestamp
0,431,2863,4.0,1165548515
1,571,7173,2.0,1334343358
2,77,223,4.5,1163004353
3,580,1032,4.0,1165291033
4,624,1221,5.0,1019124147
...,...,...,...,...
74998,547,5810,3.0,1415444349
74999,418,1835,4.0,1132180632
75000,5,33679,4.0,1163374517
75001,358,905,5.0,957479957


In [20]:
smd['genres'] = [list(genres.transform(sublist)) for sublist in smd['genres']]

In [187]:
smd['keywords'] = [list(keywords.transform(sublist)) for sublist in smd['keywords']]

smd['spoken_languages'] = [list(sp_lan.transform(sublist)) for sublist in smd['spoken_languages']]

smd['production_companies'] = [list(pro_com.transform(sublist)) for sublist in smd['production_companies']]

smd['production_countries'] = [list(pro_coun.transform(sublist)) for sublist in smd['production_countries']]

In [188]:
smd

,id,movieId,title,genres,description,keywords,cast,director,spoken_languages,production_companies,production_countries,popularity,year,vote_average,vote_count,wr
0,862,1,Toy Story,"[2, 3, 7]","Led by Woody, Andy's toys live happily in his ...","[3045, 5932, 691, 2285, 2284, 4878, 700, 3978,...","[tomhanks, timallen, donrickles]",1706,[12],[5110],[91],21.946943,1995,7,5415,6.869770
1,8844,2,Jumanji,"[1, 8, 7]",When siblings Judy and Peter discover an encha...,"[630, 1600, 448, 3957, 4740, 2407]","[robinwilliams, jonathanhyde, kirstendunst]",1631,"[12, 15]","[6722, 6404, 3270]",[91],17.015539,1995,6,2413,5.884891
2,15602,3,Grumpier Old Men,"[14, 3]",A family wedding reignites the ancient feud be...,"[2178, 528, 1750]","[waltermatthau, jacklemmon, ann-margret]",1322,[12],"[7108, 3655]",[91],11.712900,1995,6,92,5.376968
3,31357,4,Waiting to Exhale,"[3, 6, 14]","Cheated on, mistreated and stepped on, the wom...","[457, 2963, 5294, 1637, 1020]","[whitneyhouston, angelabassett, lorettadevine]",1010,[12],[6785],[91],3.859495,1995,6,34,5.299755
4,11862,5,Father of the Bride Part II,[3],Just when George Banks has recovered from his ...,"[370, 3657, 1234, 72, 1468, 3793, 4504, 1260, ...","[stevemartin, dianekeaton, martinshort]",500,[12],"[5717, 6691]",[91],8.387519,1995,5,173,5.175099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9214,159550,161944,The Last Brickmaker in America,[6],A man must cope with the loss of his wife and ...,[2285],"[sidneypoitier, wendycrewson, jayo.sanders]",1204,[],[4508],[91],0.038998,2001,7,1,5.248931
9215,392572,162542,Rustom,"[17, 14]","Rustom Pavri, an honourable officer of the Ind...",[650],"[akshaykumar, ileanad'cruz, eshagupta]",3366,[56],[3585],[35],7.333139,2016,7,25,5.340490
9216,402672,162672,Mohenjo Daro,"[1, 6, 10, 14]","Village lad Sarman is drawn to big, bad Mohenj...",[650],"[hrithikroshan, poojahegde, kabirbedi]",243,[56],"[6830, 471]",[35],1.423358,2016,6,26,5.287576
9217,315011,163056,Shin Godzilla,"[0, 1, 6, 11, 15]",From the mind behind Evangelion comes a hit la...,"[3759, 2444, 2409, 1563, 3111]","[hirokihasegawa, yutakatakenouchi, satomiishih...",1304,"[19, 10, 12, 63]","[1281, 6648]",[42],9.285519,2016,6,152,5.440760


In [176]:
def get_sim_movie (target, lists_enc, k ):
    cosine_sim_values = []
    for list_enc in lists_enc:
        if len(list_enc) == len(target):
            list_vec_1s = np.array(list_enc)
            list_vec_2s = np.array(target)
            list_vec_1s = list_vec_1s.reshape(1, -1)
            list_vec_2s = list_vec_2s.reshape(1, -1)
            list_sims = cosine_similarity(list_vec_1s,list_vec_2s)
            cosine_sim_values.append((list_sims,list_enc))
        else:
            pass
    sorted_sim_values = sorted(cosine_sim_values, key=lambda x: x[0], reverse=True)
    most_sim_movies = [list_enc[1] for list_enc in sorted_sim_values[:k]]
    return most_sim_movies

In [190]:
def rec_movie_genre (user_id):
    movies_id = train_df[train_df.userId == user_id].movieId
    recommended = []
    recommended = pd.DataFrame(recommended)
    for movie in movies_id:
        movie_list = smd[smd['movieId'] == movie]
        movies_genre = movie_list['genres'].values.tolist()
    for movie_genre in movies_genre:
        most_sim_movies = list(get_sim_movie ( movie_genre, genres_enc, k =10))
    for sim_movie in most_sim_movies:
        for index, row in smd.iterrows():
            if all(elem in row['genres'] for elem in sim_movie):
        # Add the row to the filtered DataFrame
                recommended = recommended.append(row)
    recommended['userId'] = user_id
    recommended['movieId'] = recommended['movieId'].astype('int64')
    recommended = recommended[['userId','movieId']]
    return recommended

In [191]:
rec_df_genre = rec_movie_genre (73)

In [192]:
rec_df_genre

,userId,movieId
6787,73,53468
7185,73,64900
6516,73,46062
7145,73,63393
7171,73,64338
...,...,...
8438,73,103449
8831,73,118354
8925,73,129937
9043,73,139644


In [209]:
def rec_movie_language (user_id):
    movies_id = train_df[train_df.userId == user_id].movieId
    recommended = []
    recommended = pd.DataFrame(recommended)
    for movie in movies_id:
        movie_list = smd[smd['movieId'] == movie]
        movies_language = movie_list['spoken_languages'].values.tolist()
    for movie_language in movies_language:
        most_sim_movies = list(get_sim_movie ( movie_language, spoken_languages_enc, k =5))
    for sim_movie in most_sim_movies:
        for index, row in smd.iterrows():
            if all(elem in row['spoken_languages'] for elem in sim_movie):
        # Add the row to the filtered DataFrame
                recommended = recommended.append(row)
    recommended['userId'] = user_id
    recommended['movieId'] = recommended['movieId'].astype('int64')
    recommended = recommended[['userId','movieId']]
    return recommended

In [210]:
rec_df_language = rec_movie_language (73)

In [193]:
def rec_movie_keyword (user_id):
    movies_id = train_df[train_df.userId == user_id].movieId
    recommended = []
    recommended = pd.DataFrame(recommended)
    for movie in movies_id:
        movie_list = smd[smd['movieId'] == movie]
        movies_keyword = movie_list['keywords'].values.tolist()
    for movie_keyword in movies_keyword:
        most_sim_movies = list(get_sim_movie ( movie_keyword, keyword_enc, k =10))
    for sim_movie in most_sim_movies:
        for index, row in smd.iterrows():
            if all(elem in row['keywords'] for elem in sim_movie):
        # Add the row to the filtered DataFrame
                recommended = recommended.append(row)
    recommended['userId'] = user_id
    recommended['movieId'] = recommended['movieId'].astype('int64')
    recommended = recommended[['userId','movieId']]
    return recommended

In [194]:
rec_df_keyword = rec_movie_keyword (73)

In [196]:
rec_df_keyword

,userId,movieId
6787,73,53468
7377,73,70663
8419,73,103042
8447,73,103688
745,73,913
5407,73,8188
7263,73,67788
8401,73,102445
6413,73,43376
5796,73,26122


In [199]:
def rec_movie_comp (user_id):
    movies_id = train_df[train_df.userId == user_id].movieId
    recommended = []
    recommended = pd.DataFrame(recommended)
    for movie in movies_id:
        movie_list = smd[smd['movieId'] == movie]
        movies_comp = movie_list['production_companies'].values.tolist()
    for movie_comp in movies_comp:
        most_sim_movies = list(get_sim_movie ( movie_comp, production_companies_enc, k =10))
    for sim_movie in most_sim_movies:
        for index, row in smd.iterrows():
            if all(elem in row['production_companies'] for elem in sim_movie):
        # Add the row to the filtered DataFrame
                recommended = recommended.append(row)
    recommended['userId'] = user_id
    recommended['movieId'] = recommended['movieId'].astype('int64')
    recommended = recommended[['userId','movieId']]
    return recommended

In [200]:
rec_df_comp = rec_movie_comp (73)

In [201]:
rec_df_comp

,userId,movieId
6787,73,53468
2571,73,3178
3712,73,4700
8543,73,106766
71,73,77
6210,73,33669
957,73,1181
5973,73,27555
6525,73,46574
8218,73,96815


In [207]:
def rec_movie_country (user_id):
    movies_id = train_df[train_df.userId == user_id].movieId
    recommended = []
    recommended = pd.DataFrame(recommended)
    for movie in movies_id:
        movie_list = smd[smd['movieId'] == movie]
        movies_country = movie_list['production_countries'].values.tolist()
    for movie_country in movies_country:
        most_sim_movies = list(get_sim_movie ( movie_country, production_countries_enc, k =5))
    for sim_movie in most_sim_movies:
        for index, row in smd.iterrows():
            if all(elem in row['production_countries'] for elem in sim_movie):
        # Add the row to the filtered DataFrame
                recommended = recommended.append(row)
    recommended['userId'] = user_id
    recommended['movieId'] = recommended['movieId'].astype('int64')
    recommended = recommended[['userId','movieId']]
    return recommended

In [208]:
rec_df_country = rec_movie_country (73)

In [213]:
smd['combined'] = smd['keywords']+smd['production_companies']

In [214]:
combined_enc = smd['combined'].values.tolist()

In [219]:
def rec_movie_combined (user_id):
    movies_id = train_df[train_df.userId == user_id].movieId
    recommended = []
    recommended = pd.DataFrame(recommended)
    for movie in movies_id:
        movie_list = smd[smd['movieId'] == movie]
        movies_comb = movie_list['combined'].values.tolist()
    for movie_comb in movies_comb:
        most_sim_movies = list(get_sim_movie ( movie_comb, combined_enc, k =5))
    for sim_movie in most_sim_movies:
        for index, row in smd.iterrows():
            if all(elem in row['combined'] for elem in sim_movie):
        # Add the row to the filtered DataFrame
                recommended = recommended.append(row)
    recommended['userId'] = user_id
    recommended['movieId'] = recommended['movieId'].astype('int64')
    recommended = recommended[['userId','movieId']]
    return recommended

In [220]:
rec_df_comb = rec_movie_combined (73)

# EVALUATE ON TEST SET

In [153]:
from evaluate_model import evaluate

In [145]:
testset = pd.read_csv('test_set.csv')

In [146]:
testset = testset[['userId', 'movieId']]
testset

,userId,movieId
0,302,593
1,191,110
2,457,5337
3,239,1042
4,292,5377
...,...,...
24996,353,34319
24997,428,743
24998,595,1097
24999,664,61248


In [185]:
evaluate(rec_df_genre, testset)

0.017811704834605598

In [204]:
evaluate(rec_df_comp, testset)

0.1

In [205]:
evaluate(rec_df_keyword, testset)

0.1

In [211]:
evaluate(rec_df_country, testset)

0.0502679994205418

In [212]:
evaluate(rec_df_language, testset)

0.04705446461652464

In [222]:
evaluate(rec_df_comb, testset)

0.2